In [5]:
##
##Script to get List of categories available in Yelp that list restraurants in ranking
##
import pandas as pd
import numpy as np
#for creating geo coordinate combinations
import itertools  as it
from yelpapi import YelpAPI
import requests     # how python goes onto the internet!
import re           # regex
from bs4 import BeautifulSoup # a python HTML parser (version 3)
from urllib2 import Request, urlopen
import json
from pandas.io.json import json_normalize
%matplotlib inline
from pprint import pprint
import time

#### https://www.yelp.com/developers/v2/manage_api_keys refresh to generate new keys

In [6]:
api_key = 'SD5ybcet2QSwWXz2N7M5lw'
api_secret = 'EJBXJBAFgEka0br1ynT6HwWJVT8'
access_token = 'EESADq5kSfBL4XkbxzwJxsqhZIgQHbJs'
access_secret = 'JMU-eh8i0eVUUYxJcjelLkZEfTY'

#### activate yelpapi with personal keys

In [7]:
yelp_api = YelpAPI(api_key, api_secret, access_token, access_secret)

#### Build index of all the field we care about

In [8]:
index = {'id':'True','is_closed':'True','name':'True','image_url':'True', 'url':'True','mobile_url':'True','phone':'True', 'display_phone':'True', 'review_count':'True','categories':'True', 'rating':'True', 'snippet_text':'True', 'location_address':'True', 'location_display_address':'True', 'location_city':'True','location_state_code':'True','location_postal_code':'True','location_country_code':'True','location_cross_streets':'True','location_cross_streets':'True','location_neighborhoods':'True','location_coordinate':'True',}
index

{'categories': 'True',
 'display_phone': 'True',
 'id': 'True',
 'image_url': 'True',
 'is_closed': 'True',
 'location_address': 'True',
 'location_city': 'True',
 'location_coordinate': 'True',
 'location_country_code': 'True',
 'location_cross_streets': 'True',
 'location_display_address': 'True',
 'location_neighborhoods': 'True',
 'location_postal_code': 'True',
 'location_state_code': 'True',
 'mobile_url': 'True',
 'name': 'True',
 'phone': 'True',
 'rating': 'True',
 'review_count': 'True',
 'snippet_text': 'True',
 'url': 'True'}

## Create a method to grab yelp business with a keyword in a lat-long bound

In [9]:
#Create method to return json to row
def parse_restaurant(rest_json):
    index = {'id':'True','is_closed':'True','name':'True','image_url':'True', 'url':'True','mobile_url':'True','phone':'True', 'display_phone':'True', 'review_count':'True','categories':'True', 'rating':'True', 'snippet_text':'True', 'location_address':'True', 'location_display_address':'True', 'location_city':'True','location_state_code':'True','location_postal_code':'True','location_country_code':'True','location_cross_streets':'True','location_neighborhoods':'True','location_coordinate':'True'}
    if 'id' in index: 
        _id = rest_json.get('id')
    else: _id = 'NaN'
    if 'is_closed' in index: 
        is_closed = rest_json.get('is_closed')
    else: is_closed = 'NaN'
    if 'name' in index:
        name = rest_json.get('name')
    else: name = 'NaN'
    if 'image_url' in index:
        image_url = rest_json.get('image_url')
    else: image_url = 'NaN'
    if 'url' in index:
        url = rest_json.get('url')
    else: url = 'NaN'
    if 'mobile_url' in index:
        mobile_url = rest_json.get('mobile_url')
    else: mobile_url = 'NaN'
    if 'phone' in index:
        phone = rest_json.get('phone')
    else: phone = 'NaN'
    if 'display_phone' in index:
        display_phone = rest_json.get('display_phone')
    else: display_phone = 'NaN'
    if 'review_count' in index:
        review_count = rest_json.get('review_count')
    else: review_count = 'NaN'
    if 'categories' in index:
        categories = rest_json.get('categories')
    else: categories = 'NaN'
    if 'rating' in index:
        rating = rest_json.get('rating')
    else: rating = 'NaN'
    if 'snippet_text' in index:
        snippet_text = rest_json.get('snippet_text')
    else: snippet_text = 'NaN'
    if 'location_address' in index:
        location_address = rest_json['location'].get('address')
    else: location_address = 'NaN'
    if 'location_display_address' in index:
        location_display_address = rest_json['location'].get('display_address')
    else: location_display_address = 'NaN'
    if 'location_city' in index:
        location_city = rest_json['location'].get('city')
    else: location_city = 'NaN'
    if 'location_state_code' in index:
        location_state_code = rest_json['location'].get('state_code')
    else: location_state_code= 'Nan'        
    if 'location_postal_code' in index:
        location_postal_code = rest_json['location'].get('postal_code')
    else: location_postal_code = 'Nan'
    if 'location_country_code' in index:
        location_country_code = rest_json['location'].get('country_code')
    else: location_country_code = 'Nan'        
    if 'location_cross_streets' in index:
        location_cross_streets = rest_json['location'].get('cross_streets')
    else: location_cross_streets = 'Nan'
    if 'location_neighborhoods' in index:
        location_neighborhoods = rest_json['location'].get('neighborhoods')
    else: location_neighborhoods = 'Nan'
    if 'location_coordinate' in index:
        location_coordinate = rest_json['location'].get('coordinate') 
    else: location_coordinate= 'Nan'
    #good thing about get function is that it returns if there is a result and doesn't break if there isn't
    return [_id,is_closed, name,image_url,url,mobile_url,phone,display_phone,review_count,categories,rating,snippet_text,location_address, location_display_address, location_city,location_state_code,location_postal_code,location_country_code,location_cross_streets,location_neighborhoods,location_coordinate]

In [10]:
#Create method to call yelp api with term and bound filters
def getbiz(term, bounds):
    search_results_df = pd.DataFrame({})
    offset = 0
    while offset<= 980:
        temp_dict = yelp_api.search_query(term= term, bounds=bounds, offset=offset)
        rows_of_df = []
        for bar_json in temp_dict['businesses']:
            new_row = parse_restaurant(bar_json)
            rows_of_df.append(new_row)
        temp_df = pd.DataFrame(rows_of_df, columns=['id','is_closed','name','image_url', 'url','mobile_url','phone', 'display_phone', 'review_count','categories', 'rating', 'snippet_text', 'location_address', 'location_display_address', 'location_city','location_state_code','location_postal_code','location_country_code','location_cross_streets','location_neighborhoods','location_coordinate'])
        search_results_df = search_results_df.append(temp_df, ignore_index=True)
        if temp_df.shape[0]<20:
            print 'Results exhausted at '+ str(temp_df.shape[0]+offset)+' results at offset '+str(offset)
            break;
        offset += 20
    return search_results_df;

In [11]:
#Function to get 1 if a number is not a multiple of something 
import math
def rem(x,y):
    res = x % y
    return math.copysign(res,x)


In [12]:
#create method to get list of bounds

def get_bounds(nw_latitude, nw_longitude, se_latitude, se_longitude,block):
    #Specify how many rows and columns you wish to devide the geographical bounds into
    #block = 15
    increment_lat = (nw_latitude-se_latitude)/block
    increment_lon = (nw_longitude-se_longitude)/block

    #Find latitude and longitude blocks coordinates
    lat = np.arange(se_latitude, nw_latitude, increment_lat)
    lon = np.arange(se_longitude, nw_longitude, increment_lon)

    #build a block squared list of coordinates
    import itertools
    listoflist = lat,lon

    #make all combinations of bounds required for calculation
    comb  = list(itertools.product(*listoflist))

    #sequence the list of coordinates to run north (east to west) to south (east to west)
    comb = comb[::-1]
    
    i=0
    bounds = []
    #print (block*(block-1)-1)
    while i<=(block*(block-1)-1):
    #while i<=block*block:
        if rem(i,block) != (block-1):
            bounds = bounds + [str(comb[i][0])+','+str(comb[i][1])+'|'+str(comb[i+1+block][0])+','+str(comb[i+1+block][1])]
            #print len(bounds)
            i += 1
        elif rem(i,block) == (block-1):
            i += 1
    return bounds;
    


In [13]:
def getall(term, nw_latitude, nw_longitude, se_latitude, se_longitude, block):
    bounds = get_bounds(nw_latitude, nw_longitude, se_latitude, se_longitude, block)
    #Run getbiz method for each element in bounds
    b = 0
    biz=pd.DataFrame({})
    while b <= (len(bounds)-1):
        biz = biz.append(getbiz(term,bounds[b]), ignore_index=True)
        #print b
        b+=1
    print biz.shape
    return biz;

In [14]:
#Specify the keyword to search
term = 'bar'

#Specify the lat lon to specify bounds
nw_latitude=37.810298
nw_longitude=-122.522450
se_latitude=37.704093
se_longitude=-122.375803
block = 20

#Get bounds list using method
#bounds = get_bounds(nw_latitude, nw_longitude, se_latitude, se_longitude, block)

#Get
bars = getall(term,nw_latitude, nw_longitude, se_latitude, se_longitude, block)
bars['location_latitude'] = bars['location_coordinate'].apply(lambda x:'unknown' if x == None else x.get('latitude'))
bars['location_longitude'] = bars['location_coordinate'].apply(lambda x:'unknown' if x == None else x.get('longitude'))



Results exhausted at 0 results at offset 0
Results exhausted at 0 results at offset 0
Results exhausted at 0 results at offset 0
Results exhausted at 0 results at offset 0
Results exhausted at 0 results at offset 0
Results exhausted at 3 results at offset 0
Results exhausted at 3 results at offset 0
Results exhausted at 3 results at offset 0
Results exhausted at 1 results at offset 0
Results exhausted at 4 results at offset 0
Results exhausted at 30 results at offset 20
Results exhausted at 4 results at offset 0
Results exhausted at 6 results at offset 0
Results exhausted at 12 results at offset 0
Results exhausted at 32 results at offset 20
Results exhausted at 9 results at offset 0
Results exhausted at 1 results at offset 0
Results exhausted at 0 results at offset 0
Results exhausted at 0 results at offset 0
Results exhausted at 0 results at offset 0
Results exhausted at 0 results at offset 0
Results exhausted at 0 results at offset 0
Results exhausted at 0 results at offset 0
Result

In [15]:
print bars.shape
#biz = wine.append(bars, ignore_index=True)
#print biz.shape
#biz.drop_duplicates(['id', 'name', 'location_coordinate'], 'Firse')

(2590, 23)


In [134]:
bars.head(50)

,id,is_closed,name,image_url,url,mobile_url,phone,display_phone,review_count,categories,...,location_display_address,location_city,location_state_code,location_postal_code,location_country_code,location_cross_streets,location_neighborhoods,location_coordinate,location_latitude,location_longitude
0,log-cabin-at-the-presidio-san-francisco,False,Log Cabin at the Presidio,https://s3-media3.fl.yelpcdn.com/bphoto/CmBexe...,https://www.yelp.com/biz/log-cabin-at-the-pres...,https://m.yelp.com/biz/log-cabin-at-the-presid...,4155615444,+1-415-561-5444,25.0,"[[Venues & Event Spaces, venues]]",...,"[1299 Storey Ave, Presidio, San Francisco, CA ...",San Francisco,CA,94129,US,Miller Rd & Stone St,[Presidio],"{u'latitude': 37.8038376838, u'longitude': -12...",37.8038,-122.473
1,presidio-nursery-san-francisco,False,Presidio Nursery,https://s3-media1.fl.yelpcdn.com/bphoto/qKvyb1...,https://www.yelp.com/biz/presidio-nursery-san-...,https://m.yelp.com/biz/presidio-nursery-san-fr...,4155614826,+1-415-561-4826,5.0,"[[Nurseries & Gardening, gardening]]",...,"[1249 Appleton St, Presidio, San Francisco, CA...",San Francisco,CA,94129,US,None,[Presidio],"{u'latitude': 37.80084332, u'longitude': -122....",37.8008,-122.471
2,balance-san-francisco,False,Balance,https://s3-media1.fl.yelpcdn.com/bphoto/V-6nW9...,https://www.yelp.com/biz/balance-san-francisco...,https://m.yelp.com/biz/balance-san-francisco?a...,4407594347,+1-440-759-4347,5.0,"[[Fitness & Instruction, fitness]]",...,"[Presidio, San Francisco, CA 94129]",San Francisco,CA,94129,US,None,[Presidio],"{u'latitude': 37.8011017, u'longitude': -122.4...",37.8011,-122.472
3,fort-scott-field-at-the-presidio-san-francisco,False,Fort Scott Field at the Presidio,https://s3-media4.fl.yelpcdn.com/bphoto/LZVhxP...,https://www.yelp.com/biz/fort-scott-field-at-t...,https://m.yelp.com/biz/fort-scott-field-at-the...,4155614200,+1-415-561-4200,6.0,"[[Recreation Centers, recreation], [Soccer, fo...",...,"[Presidio, San Francisco, CA 94129]",San Francisco,CA,94129,US,None,[Presidio],"{u'latitude': 37.8031542208, u'longitude': -12...",37.8032,-122.475
4,roaring-mouse-cycles-san-francisco,False,Roaring Mouse Cycles,https://s3-media2.fl.yelpcdn.com/bphoto/PrUfCR...,https://www.yelp.com/biz/roaring-mouse-cycles-...,https://m.yelp.com/biz/roaring-mouse-cycles-sa...,4157536272,+1-415-753-6272,209.0,"[[Bikes, bikes], [Bike Repair/Maintenance, bik...",...,"[934 Old Mason St, Presidio, San Francisco, CA...",San Francisco,CA,94129,US,None,[Presidio],"{u'latitude': 37.8040876459, u'longitude': -12...",37.8041,-122.468
5,house-of-air-san-francisco,False,House of Air,https://s3-media3.fl.yelpcdn.com/bphoto/0L-53c...,https://www.yelp.com/biz/house-of-air-san-fran...,https://m.yelp.com/biz/house-of-air-san-franci...,4152003068,+1-415-200-3068,417.0,"[[Recreation Centers, recreation], [Trampoline...",...,"[926 Mason St, Presidio, San Francisco, CA 94129]",San Francisco,CA,94129,US,None,[Presidio],"{u'latitude': 37.804902, u'longitude': -122.46...",37.8049,-122.469
6,planet-granite-san-francisco-2,False,Planet Granite,https://s3-media2.fl.yelpcdn.com/bphoto/jJmMYr...,https://www.yelp.com/biz/planet-granite-san-fr...,https://m.yelp.com/biz/planet-granite-san-fran...,4156923434,+1-415-692-3434,365.0,"[[Gyms, gyms], [Yoga, yoga], [Rock Climbing, r...",...,"[924 Mason St, Presidio, San Francisco, CA 94...",San Francisco,CA,94129,US,None,[Presidio],"{u'latitude': 37.8042571852, u'longitude': -12...",37.8043,-122.468
7,he-commissary-san-francisco,False,The Commissary,https://s3-media4.fl.yelpcdn.com/bphoto/srMW70...,https://www.yelp.com/biz/he-commissary-san-fra...,https://m.yelp.com/biz/he-commissary-san-franc...,4155613600,+1-415-561-3600,257.0,"[[Spanish, spanish], [American (New), newameri...",...,"[101 Montgomery St, Presidio, San Francisco, C...",San Francisco,CA,94129,US,None,[Presidio],"{u'latitude': 37.799758721, u'longitude': -122...",37.7998,-122.46
8,american-lamb-jam-san-francisco,False,American Lamb Jam,https://s3-media3.fl.

In [16]:
###### Export to csv
#Previous = pd.read_csv('../data/yelp_businesses.csv', parse_dates=True)
bars.to_csv('../data/yelp_businesses.csv',encoding='utf-8')
#wine.to_csv('../data/yelp_businesses2.csv',encoding='utf-8')

### Add hours and price info

In [17]:
bars = pd.read_csv('../data/yelp_businesses.csv', parse_dates=True)
#bars = pd.DataFrame(bars.id)
bars.head()

,Unnamed: 0,id,is_closed,name,image_url,url,mobile_url,phone,display_phone,review_count,...,location_display_address,location_city,location_state_code,location_postal_code,location_country_code,location_cross_streets,location_neighborhoods,location_coordinate,location_latitude,location_longitude
0,0,log-cabin-at-the-presidio-san-francisco,False,Log Cabin at the Presidio,https://s3-media3.fl.yelpcdn.com/bphoto/CmBexe...,https://www.yelp.com/biz/log-cabin-at-the-pres...,https://m.yelp.com/biz/log-cabin-at-the-presid...,4.155615e+09,+1-415-561-5444,24.0,...,"[1299 Storey Ave, Presidio, San Francisco, CA ...",San Francisco,CA,94129.0,US,Miller Rd & Stone St,[Presidio],"{u'latitude': 37.8038376838, u'longitude': -12...",37.8038376838,-122.473276203
1,1,presidio-nursery-san-francisco,False,Presidio Nursery,https://s3-media1.fl.yelpcdn.com/bphoto/qKvyb1...,https://www.yelp.com/biz/presidio-nursery-san-...,https://m.yelp.com/biz/presidio-nursery-san-fr...,4.155615e+09,+1-415-561-4826,5.0,...,"[1249 Appleton St, Presidio, San Francisco, CA...",San Francisco,CA,94129.0,US,NaN,[Presidio],"{u'latitude': 37.80084332, u'longitude': -122....",37.80084332,-122.4711798
2,2,fort-scott-field-at-the-presidio-san-francisco,False,Fort Scott Field at the Presidio,https://s3-media4.fl.yelpcdn.com/bphoto/LZVhxP...,https://www.yelp.com/biz/fort-scott-field-at-t...,https://m.yelp.com/biz/fort-scott-field-at-the...,4.155614e+09,+1-415-561-4200,5.0,...,"[Presidio, San Francisco, CA 94129]",San Francisco,CA,94129.0,US,NaN,[Presidio],"{u'latitude': 37.8031542208, u'longitude': -12...",37.8031542208,-122.474578895
3,3,roaring-mouse-cycles-san-francisco,False,Roaring Mouse Cycles,https://s3-media2.fl.yelpcdn.com/bphoto/PrUfCR...,https://www.yelp.com/biz/roaring-mouse-cycles-...,https://m.yelp.com/biz/roaring-mouse-cycles-sa...,4.157536e+09,+1-415-753-6272,209.0,...,"[934 Old Mason St, Presidio, San Francisco, CA...",San Francisco,CA,94129.0,US,NaN,[Presidio],"{u'latitude': 37.8040876459, u'longitude': -12...",37.8040876459,-122.467603683
4,4,house-of-air-san-francisco,False,House of Air,https://s3-media3.fl.yelpcdn.com/bphoto/0L-53c...,https://www.yelp.com/biz/house-of-air-san-fran...,https://m.yelp.com/biz/house-of-air-san-franci...,4.152003e+09,+1-415-200-3068,416.0,...,"[926 Mason St, Presidio, San Francisco, CA 94129]",San Francisco,CA,94129.0,US,NaN,[Presidio],"{u'latitude': 37.804902, u'longitude': -122.46...",37.804902,-122.468805


In [18]:
type(bars)

pandas.core.frame.DataFrame

### Yelp API for getting Price and Hours Info

In [19]:
client_id = '20CN1D1j-IjFTxtPlDFZhQ'
client_secret = '2ACsEGrBzN9KLxiusGrUk7KDhULH6j9WEendPpZeMLzm8ng5vurL2Vy14kO8ARy3'
re = requests.post('https://api.yelp.com/oauth2/token', data = {'grant_type':'client_credentials', 'client_id':client_id, 'client_secret':client_secret})

In [20]:
#eval(re.content)['access_token']
eval(re.content)

{'access_token': 'ZtzpCWS-VsT09K0muYymeOPsW07Xr3OiBRl0rj3c37f0CEqf9fyMZhTlvM0ksZYMgexT-YdhuGI0yYKRPceZGaMMFEF8Di-t5u2nRC4n5eph4iUF78itWp7MUZUOWHYx',
 'expires_in': 11917346,
 'token_type': 'Bearer'}

In [21]:
access_token = eval(re.content)['access_token']
token_type = eval(re.content)['token_type']
headers = token_type + ' ' + access_token
expires_in = eval(re.content)['expires_in']
#re = requests.head('https://api.yelp.com/oauth2/'+access_token)
headers = {"Authorization": headers}
endpoint = 'https://api.yelp.com/v3/businesses/'

### Test with sample business ID

In [22]:
id = 'north-india-restaurant-san-francisco'
a =  requests.get(endpoint+id, headers = headers).content
print a
print '----'
hours = json.loads(a)['hours'][0].get('open')
price = json.loads(a)['price'][0]
print pprint(hours)
print price

{"coordinates": {"latitude": 37.787789124691, "longitude": -122.399305736113}, "photos": ["https://s3-media1.fl.yelpcdn.com/bphoto/howYvOKNPXU9A5KUahEXLA/o.jpg", "https://s3-media2.fl.yelpcdn.com/bphoto/I-CX8nioj3_ybAAYmhZcYg/o.jpg", "https://s3-media3.fl.yelpcdn.com/bphoto/uaSNfzJUiFDzMeSCwTcs-A/o.jpg"], "url": "https://www.yelp.com/biz/north-india-restaurant-san-francisco?adjust_creative=20CN1D1j-IjFTxtPlDFZhQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=20CN1D1j-IjFTxtPlDFZhQ", "id": "north-india-restaurant-san-francisco", "location": {"zip_code": "94105", "address3": "", "city": "San Francisco", "address2": "", "country": "US", "address1": "123 Second St", "state": "CA"}, "name": "North India Restaurant", "hours": [{"is_open_now": true, "hours_type": "REGULAR", "open": [{"day": 0, "start": "1000", "end": "2300", "is_overnight": false}, {"day": 1, "start": "1000", "end": "2300", "is_overnight": false}, {"day": 2, "start": "1000", "end": "2300", "is_overnight

### Loop through all businesses to get all info

In [35]:
#This works!!
import datetime
print datetime.datetime.now()
rows_of_df = []
for bar_json in bars['id']:
    new_row = requests.get(endpoint+bar_json, headers = headers).content
    rows_of_df.append(new_row)
    time.sleep(time.localtime(time.time())[5]/15.0)
print len(rows_of_df)
#This works!!
print datetime.datetime.now()

2016-12-05 17:05:39.832284


SSLError: EOF occurred in violation of protocol (_ssl.c:590)

344

In [ ]:
rows_of_df = pd.Series(rows_of_df)
bars['added_info']= rows_of_df.values

### Parse hours from Added Info

In [ ]:
bars['hours'] = bars['added_info'].apply(lambda x: json.loads(str(x)).get('hours') if is_json(x) != 0 else '')

### Export updated csv

In [68]:

bars.to_csv('../data/yelp_businesses.csv',encoding='utf-8')

### Clean up new info and add columns

In [47]:
bars.shape

(2527, 28)

In [54]:
hours = []
for i in bars['added_info']:
    if i != None and json.loads(i).get('hours') != None:
        hours += json.loads(str(i)).get('hours')
    else: hours += ''
hours

ValueError: No JSON object could be decoded

In [35]:
bars['hours'] = bars['added_info'].apply(lambda x:'unknown' if x == None and x.notnull() else json.loads(str(x)).get('hours'))

ValueError: No JSON object could be decoded

In [250]:
hours

[]

### Parse new column to seperate fields

In [197]:
json.loads(bars['added_info'][300]).get('hours')
#bars['hours']=bars['added_info'].apply(lambda x:json.loads(x)[0].get('hours'))
#bars['hours_type']=bars['hours'].apply(lambda x:x[0].get('hours_type'))
#bars['hours_open']=bars['hours'].apply(lambda x:x[0].get('open'))
#bars['price']=bars['added_info'].apply(lambda x:json.loads(x).get('price'))

[{u'hours_type': u'REGULAR',
  u'is_open_now': False,
  u'open': [{u'day': 0,
    u'end': u'2200',
    u'is_overnight': False,
    u'start': u'1130'},
   {u'day': 1, u'end': u'2200', u'is_overnight': False, u'start': u'1130'},
   {u'day': 2, u'end': u'2200', u'is_overnight': False, u'start': u'1130'},
   {u'day': 3, u'end': u'2200', u'is_overnight': False, u'start': u'1130'},
   {u'day': 4, u'end': u'2200', u'is_overnight': False, u'start': u'1130'},
   {u'day': 5, u'end': u'2200', u'is_overnight': False, u'start': u'1130'},
   {u'day': 6, u'end': u'2200', u'is_overnight': False, u'start': u'1130'}]}]

In [128]:
#json.loads(bars['added_info'][0]).get('rating')
#print json.loads(bars['added_info'][0]).get('price')
#json.loads(bars['added_info'][300]).get('hours')[0].get('hours_type')
bars['hours']=bars['added_info'].apply(lambda x:json.loads(x).get('hours'))

ValueError: No JSON object could be decoded

In [643]:
#bars = bars['id']
bars['bizinfo']=bars['id'].apply(lambda x: requests.get(endpoint+x, headers = headers).content)
bars['hours']=bars['bizinfo'].apply(lambda x:json.loads(x).get('hours'))
bars['hours_type']=bars['hours'].apply(lambda x:x[0].get('hours_type'))
bars['hours_open']=bars['hours'].apply(lambda x:x[0].get('open'))
bars['price']=bars['bizinfo'].apply(lambda x:json.loads(x).get('price'))
#bars.bizinfo
#wine['location_longitude'] = wine['location_coordinate'].apply(lambda x:'unknown' if x == None else x.get('longitude'))

In [670]:
bars.head()

,Unnamed: 0,id,is_closed,name,image_url,url,mobile_url,phone,display_phone,review_count,...,location_cross_streets,location_neighborhoods,location_coordinate,location_latitude,location_longitude,bizinfo,hours,hours_type,hours_open,price
0,0,log-cabin-at-the-presidio-san-francisco,False,Log Cabin at the Presidio,https://s3-media3.fl.yelpcdn.com/bphoto/CmBexe...,https://www.yelp.com/biz/log-cabin-at-the-pres...,https://m.yelp.com/biz/log-cabin-at-the-presid...,4.155615e+09,+1-415-561-5444,25.0,...,Miller Rd & Stone St,[Presidio],"{u'latitude': 37.8038376838, u'longitude': -12...",37.8038376838,-122.473276203,"{""image_url"": ""https://s3-media3.fl.yelpcdn.co...","[{u'hours_type': u'REGULAR', u'open': [], u'is...",REGULAR,[],None
1,1,presidio-nursery-san-francisco,False,Presidio Nursery,https://s3-media1.fl.yelpcdn.com/bphoto/qKvyb1...,https://www.yelp.com/biz/presidio-nursery-san-...,https://m.yelp.com/biz/presidio-nursery-san-fr...,4.155615e+09,+1-415-561-4826,5.0,...,NaN,[Presidio],"{u'latitude': 37.80084332, u'longitude': -122....",37.80084332,-122.4711798,"{""is_claimed"": true, ""phone"": ""+14155614826"", ...","[{u'hours_type': u'REGULAR', u'open': [], u'is...",REGULAR,[],$
2,2,balance-san-francisco,False,Balance,https://s3-media1.fl.yelpcdn.com/bphoto/V-6nW9...,https://www.yelp.com/biz/balance-san-francisco...,https://m.yelp.com/biz/balance-san-francisco?a...,4.407594e+09,+1-440-759-4347,5.0,...,NaN,[Presidio],"{u'latitude': 37.8011017, u'longitude': -122.4...",37.8011017,-122.472199,"{""is_claimed"": true, ""phone"": ""+14407594347"", ...","[{u'hours_type': u'REGULAR', u'open': [], u'is...",REGULAR,[],None
3,3,fort-scott-field-at-the-presidio-san-francisco,False,Fort Scott Field at the Presidio,https://s3-media4.fl.yelpcdn.com/bphoto/LZVhxP...,https://www.yelp.com/biz/fort-scott-field-at-t...,https://m.yelp.com/biz/fort-scott-field-at-the...,4.155614e+09,+1-415-561-4200,6.0,...,NaN,[Presidio],"{u'latitude': 37.8031542208, u'longitude': -12...",37.8031542208,-122.474578895,"{""is_claimed"": true, ""phone"": ""+14155614200"", ...","[{u'hours_type': u'REGULAR', u'open': [], u'is...",REGULAR,[],None
4,4,roaring-mouse-cycles-san-francisco,False,Roaring Mouse Cycles,https://s3-media2.fl.yelpcdn.com/bphoto/PrUfCR...,https://www.yelp.com/biz/roaring-mouse-cycles-...,https://m.yelp.com/biz/roaring-mouse-cycles-sa...,4.157536e+09,+1-415-753-6272,209.0,...,NaN,[Presidio],"{u'latitude': 37.8040876459, u'longitude': -12...",37.8040876459,-122.467603683,"{""coordinates"": {""longitude"": -122.46760368347...","[{u'hours_type': u'REGULAR', u'open': [{u'is_o...",REGULAR,"[{u'is_overnight': False, u'end': u'1900', u'd...",$$


In [310]:
a.content

'{"url": "https://www.yelp.com/biz/north-india-restaurant-san-francisco?adjust_creative=20CN1D1j-IjFTxtPlDFZhQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=20CN1D1j-IjFTxtPlDFZhQ", "price": "$$", "photos": ["https://s3-media1.fl.yelpcdn.com/bphoto/howYvOKNPXU9A5KUahEXLA/o.jpg", "https://s3-media2.fl.yelpcdn.com/bphoto/I-CX8nioj3_ybAAYmhZcYg/o.jpg", "https://s3-media3.fl.yelpcdn.com/bphoto/uaSNfzJUiFDzMeSCwTcs-A/o.jpg"], "name": "North India Restaurant", "coordinates": {"latitude": 37.787789124691, "longitude": -122.399305736113}, "phone": "+14153481234", "categories": [{"alias": "indpak", "title": "Indian"}], "hours": [{"hours_type": "REGULAR", "open": [{"day": 0, "end": "2300", "is_overnight": false, "start": "1000"}, {"day": 1, "end": "2300", "is_overnight": false, "start": "1000"}, {"day": 2, "end": "2300", "is_overnight": false, "start": "1000"}, {"day": 3, "end": "2300", "is_overnight": false, "start": "1000"}, {"day": 4, "end": "0000", "is_overnight": fal